# 텐초 파이토치 딥러닝 특강

## Day 03

### Pretrained Model 사용 - VGG16

In [38]:
import torch
import torch.nn as nn

from torchvision.models.vgg import vgg16

In [39]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [40]:
device

'cuda:0'

In [41]:
model = vgg16(pretrained = True)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [42]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

VGG16의 classfier 부분만 해당 task에 맞게 수정해주면 된다.

In [43]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [44]:
custom_classfier = nn.Sequential(
    nn.Linear(25088, 1024),
    nn.LeakyReLU(),
    nn.Dropout(0.2),
    nn.Linear(1024, 256),
    nn.LeakyReLU(),
    nn.Dropout(0.1),
    nn.Linear(256, 10)
)

In [45]:
model.classifier = custom_classfier

In [46]:
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [47]:
import tqdm

from torchvision.datasets.cifar import CIFAR10
from torchvision.transforms import Compose, ToTensor, Resize, RandomHorizontalFlip, RandomCrop, Normalize
from torch.utils.data import DataLoader
from torch.optim.adam import Adam

In [48]:
transforms = Compose([
    Resize(224),
    RandomCrop((224, 224), padding = 4),
    RandomHorizontalFlip(p = 0.5),
    ToTensor(),
    Normalize(mean = (0.4914, 0.4822, 0.4465), std = (0.247, 0.243, 0.261))
])

In [49]:
train = CIFAR10(root = './', train = True, download = True, transform = transforms)
test = CIFAR10(root = './', train = False, download = True, transform = transforms)

Files already downloaded and verified
Files already downloaded and verified


In [50]:
tr_loader = DataLoader(train, batch_size = 32, shuffle = True)
te_loader = DataLoader(test, batch_size = 32, shuffle = False)

In [51]:
tr_loader.dataset.data.shape[0]

50000

In [52]:
lr = 1e-4
optim = Adam(model.parameters(), lr = lr)

In [ ]:
for epoch in range(15) :
  epoch_acc = 0
  epoch_loss = 0
  
  for image, label in tqdm.tqdm(tr_loader) :

    model.train()
    
    image, label = image.to(device), label.to(device)

    optim.zero_grad()

    pred = model(image)
    cls = torch.max(pred, 1)[1]

    loss = nn.CrossEntropyLoss()(pred, label)
    loss.backward()

    optim.step()

    acc = cls.eq(label.data).sum().item()
    epoch_acc += acc

  epoch_acc /= tr_loader.dataset.data.shape[0] # 50000
  print(f"{epoch + 1} Epoch Train Accuracy : {epoch_acc}")
torch.save(model.state_dict(), 'CIFAR_VGG16.pth')

In [ ]:
model.load_state_dict(torch.load('CIFAR_VGG16.pth', map_location = device))

In [55]:
corr = 0

In [ ]:
with torch.no_grad() :

  for image, label in te_loader :
    image = image.to(device)
    label = label.to(device)

    pred = model(image)
    cls = pred.data.max(1)[1]
    corr += cls.eq(label.data).sum().item()

  print(f'Accuracy of Test : {corr / len(te_loader.dataset)}')